In [1]:
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer
import sys
sys.path.append("..")
#from structformer import StructFormer
import data_penn
import data_ptb
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import tree_utils
import collections
from nltk.parse import DependencyGraph
import nltk
from torch.nn import functional as F
import transformers
from tqdm import tqdm
import data_ptb_subword
import re
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
WORD_TAGS = [
    'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN',
    'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP',
    'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
    'WP$', 'WRB'
]

detok = TreebankWordDetokenizer()

#ptb_corpus = data_ptb.Corpus('../data/penn/')

In [3]:
def filter_words(tree):
    words = []
    for w, tag in tree.pos():
      if tag in WORD_TAGS:
        w = w.lower()
        words.append(w)
    return words

In [4]:
detok.detokenize(filter_words(ptb_corpus.train_nltktrees[-3]))

NameError: name 'ptb_corpus' is not defined

In [32]:
all_train_sens = []
for tree in ptb_corpus.train_nltktrees:
    sen = detok.detokenize(filter_words(tree))
    all_train_sens.append(sen)
    
all_test_sens = []
for tree in ptb_corpus.test_nltktrees:
    sen = detok.detokenize(filter_words(tree))
    all_test_sens.append(sen)
    
all_val_sens = []
for tree in ptb_corpus.valid_nltktress:
    sen = detok.detokenize(filter_words(tree))
    all_val_sens.append(sen)

In [33]:
len(all_train_sens), len(all_test_sens), len(all_val_sens)

(41825, 2416, 1700)

In [34]:
with open('../data/penn_subword/train_filtered_detok.txt', 'w') as f:
    for sen in all_train_sens:
        f.write(sen + '\n')

with open('../data/penn_subword/test_filtered_detok.txt', 'w') as f:
    for sen in all_test_sens:
        f.write(sen + '\n')
        
with open('../data/penn_subword/val_filtered_detok.txt', 'w') as f:
    for sen in all_val_sens:
        f.write(sen + '\n')

In [36]:
old_tokenizer = transformers.AutoTokenizer.from_pretrained("roberta-base")

In [38]:
def get_training_corpus():
    global c
    c = 0
    with open('../data/penn_subword/train_filtered_detok.txt') as f:
        for line in f:
            c += 1
            yield line.strip()

In [50]:
new_tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus(), 8_000)

In [51]:
new_tokenizer.save_pretrained("ptb_filtered_lowcase_bpe_tokenizer_8k")

('ptb_filtered_lowcase_bpe_tokenizer_8k/tokenizer_config.json',
 'ptb_filtered_lowcase_bpe_tokenizer_8k/special_tokens_map.json',
 'ptb_filtered_lowcase_bpe_tokenizer_8k/vocab.json',
 'ptb_filtered_lowcase_bpe_tokenizer_8k/merges.txt',
 'ptb_filtered_lowcase_bpe_tokenizer_8k/added_tokens.json',
 'ptb_filtered_lowcase_bpe_tokenizer_8k/tokenizer.json')

In [52]:
new_tokenizer.push_to_hub("ptb_filtered_lowcase_bpe_tokenizer_8", use_temp_dir=True)

/home/omar/miniconda3/envs/structformer/lib/python3.6/site-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/omarmomen/ptb_filtered_lowcase_bpe_tokenizer_8 into local empty directory.
To https://huggingface.co/omarmomen/ptb_filtered_lowcase_bpe_tokenizer_8
   8427847..7d165c7  main -> main



'https://huggingface.co/omarmomen/ptb_filtered_lowcase_bpe_tokenizer_8/commit/7d165c7f84f6f1b95f0c4a959b2668f2d09a700a'

In [5]:
ptb_corpus = data_ptb_subword.SubWord_Corpus("omarmomen/ptb_filtered_lowcase_bpe_tokenizer_8")

In [6]:
NEG_POSS = ['n\'t', '\'s', '\'m', '\'re', '\'ve', '\'d', '\'ll', '%']

def filter_words(tree):
    words = []
    for w, tag in tree.pos():
        if tag in WORD_TAGS:
            w = w.lower()
            words.append(w)
    return words

def tree2list(tree):
    if isinstance(tree, nltk.Tree):
        if (tree.label() in WORD_TAGS):
            w = tree.leaves()[0]
            return w
        else:
            root = []
            for child in tree:
                c = tree2list(child)
                if c:
                    root.append(c)
            if len(root) > 1:
                return root
            elif len(root) == 1:
                return root[0]
    return []

def merge_neg_poss(tree, prev_root=None):
    if isinstance(tree, list):
        root = []
        for child in tree:
            if isinstance(child, str) and (child.lower() in NEG_POSS):
                if len(root) > 0: 
                    if len(root) > 1 and isinstance(root[-1], list) and isinstance(root[-2], str): # root = [First, [of, america] ] child = 's 
                        first_str = root[-2]
                        while not isinstance(root[-1], str):
                            root = root[-1]
                        root[-1] = root[-1] + child
                        temp1 = [first_str]
                        temp2 = root
                        temp1.append(temp2)
                        root = temp1
                    else:
                        while not isinstance(root[-1], str):
                            root = root[-1]
                        root[-1] = root[-1] + child
                else:
                    try:
                        while not isinstance(prev_root[-1], str):
                            prev_root = prev_root[-1]
                        prev_root[-1] = prev_root[-1] + child
                    except IndexError:
                        pass
            else:
                root.append(merge_neg_poss(child,root))
        return root
    return tree

def merge_strings_in_tree(tree, neg_poss):
    def merge_with_last_string(node, string_to_merge):
        if isinstance(node[-1], str):
            node[-1] += string_to_merge
        elif isinstance(node[-1], list):
            merge_with_last_string(node[-1], string_to_merge)

    def merge_recursive(node, prev_node=None):
        if isinstance(node, list):
            new_node = []
            for item in node:
                merged_item = merge_recursive(item, new_node if new_node else prev_node)
                if merged_item is not None:
                    new_node.append(merged_item)
            return new_node
        else:
            if node in neg_poss and prev_node is not None:
                merge_with_last_string(prev_node, node)
                return None
            else:
                return [node]

    def remove_wrapping_lists(node):
        if isinstance(node, list):
            if len(node) == 1 and isinstance(node[0], str):
                return node[0]
            else:
                return [remove_wrapping_lists(item) for item in node]
        return node

    merged_tree = merge_recursive(tree)
    return remove_wrapping_lists(merged_tree)

def convert_tree(tree, tokenizer):
    tokenized_tree = []
    for word in tree:
        if isinstance(word, str):
            subword_tokens = tokenizer.tokenize(word)
            if len(subword_tokens) > 1:
                tokenized_subwords = convert_tree(subword_tokens, tokenizer)
                tokenized_tree.extend([tokenized_subwords])
            else:
                tokenized_tree.append(word)
        else:
            tokenized_subtree = convert_tree(word, tokenizer)
            tokenized_tree.append(tokenized_subtree)
    return tokenized_tree

def tokenize_tree(tree, tokenizer, flag):
    if isinstance(tree, list):
        tokenized_tree = []
        for child in tree:
            if isinstance(child, str):
                if flag:
                    subwords = tokenizer.tokenize(child.lower())
                else:
                    subwords = tokenizer.tokenize(' '+child.lower())
                flag = False
                subwords = subwords[0] if len(subwords) == 1 else subwords
                tokenized_tree.append(subwords)
            else:
                t, flag = tokenize_tree(child, tokenizer, flag) 
                tokenized_tree.append(t)
        return tokenized_tree, flag
    return tree, flag

def count_leaves(tree):
    if isinstance(tree, list):
        count = 0
        for child in tree:
            count += count_leaves(child)
        return count
    else:
        return 1

def double_check_detokenize(sen):
    words = sen.split(' ')
    for i in range(1, len(words)):
        if words[i] in NEG_POSS:
            words[i-1] = words[i-1] + words[i]
            words[i] = ''
    sen = ' '.join(words)
    
    words = sen.strip().split(' ')
    for i in range(len(words)):
        if words[i].lower() == 'cannot':
            words[i] = 'can'
            words.insert(i+1, 'not')
        elif words[i].lower() == 'gonna':
            words[i] = 'gon'
            words.insert(i+1, 'na')
    sen = ' '.join(words)
    
    return sen.strip()

In [7]:
sen_tree = ptb_corpus.test_nltktrees[412]

In [8]:
sen  = detok.detokenize(filter_words(sen_tree))
sen = double_check_detokenize(sen)
sen
    

"superconcentrates aren't entirely new for p&g"

In [10]:
len(ptb_corpus.tokenizer.encode(sen, add_special_tokens=False))

13

In [11]:
ptb_corpus.tokenizer.tokenize(sen)

['su',
 'per',
 'con',
 'cent',
 'rates',
 'Ġaren',
 "'t",
 'Ġentirely',
 'Ġnew',
 'Ġfor',
 'Ġp',
 '&',
 'g']

In [13]:
list_tree = tree2list(sen_tree) # get a constituent tree in list form (conatining only POS words)
if isinstance(list_tree, str):
    list_tree = [list_tree]
detokenized_tree = merge_strings_in_tree(list_tree, NEG_POSS) # merge negation and possesion and special tokens 
tokenized_tree, _ = tokenize_tree(detokenized_tree, ptb_corpus.tokenizer, flag=True) # subword tokenize the tree
tokenized_tree, count_leaves(tokenized_tree) # count the number of subwords

([['su', 'per', 'con', 'cent', 'rates'],
  [['Ġaren', "'t"], ['Ġentirely', 'Ġnew'], ['Ġfor', ['Ġp', '&', 'g']]]],
 13)

In [17]:
tokenized_tree[1]

[['Ġaren', "'t"], ['Ġentirely', 'Ġnew'], ['Ġfor', ['Ġp', '&', 'g']]]

In [14]:
list_tree

['Superconcentrates', ['are', "n't", ['entirely', 'new'], ['for', 'P&G']]]

In [419]:
detokenized_tree

[['The', 'study'],
 ['shows',
  [['that',
    [[[['nearly', '40%']], ['of', ['the', 'homeless', 'population']]],
     ['is', ['made', 'up', ['of', ['women', 'and', 'children']]]]]],
   'and',
   ['that',
    [[[['only', '25%']], ['of', ['the', 'homeless']]],
     ['exhibits',
      [['some', 'combination'],
       ['of', [['drug', 'alcohol', 'and', 'mental'], 'problems']]]]]]]]]

In [25]:
loaded_checkpoint = torch.load('../trained_models/subword_structformer_in_parser_1111_44.pt')
loaded_model = loaded_checkpoint[0]

In [22]:
sen = ptb_corpus.test_sens[412]

In [24]:
sen = ptb_corpus.tokenizer.tokenize(sen)
data = torch.LongTensor([ptb_corpus.test[412]]).to(device)
pos = torch.LongTensor([list(range(len(ptb_corpus.test[412])))]).to(device)

In [26]:
_, p_dict = loaded_model(data, pos)
block = p_dict['block']
cibling = p_dict['cibling']
head = p_dict['head']
distance = p_dict['distance']
height = p_dict['height']

In [28]:




distance = distance.clone().squeeze(0).cpu().detach().numpy().tolist()
height = height.clone().squeeze(0).cpu().detach().numpy().tolist()
head = head.clone().squeeze(0).cpu().detach().numpy()
max_height = np.max(height)

parse_tree = tree_utils.build_tree(distance, sen)

model_out, _ = tree_utils.get_brackets(parse_tree)
std_out, _ = tree_utils.get_brackets(sen_tree)
overlap = model_out.intersection(std_out)

In [44]:
parse_tree[1][0][0][1]

[["'t", 'Ġentirely'], 'Ġnew']

In [45]:
overlap

set()

In [180]:
trg_file_path = '../data/ptb/test.conllu'

with open(trg_file_path, 'r') as trg_file:
    trg_string = trg_file.read().strip()
    trg_string_list = trg_string.split('\n\n')
    try:
      trg_list = [
          DependencyGraph(t, top_relation_label='root') for t in trg_string_list
      ]
    except ValueError:

      def extract_10_cells(cells, index):
        line_index, word, lemma, tag, _, head, rel, _, _, _ = cells
        try:
          index = int(line_index)
        except ValueError:
          # index can't be parsed as an integer, use default
          pass
        return index, word, lemma, tag, tag, '', head, rel

      trg_list = [
          DependencyGraph(
              t, top_relation_label='root', cell_extractor=extract_10_cells)
          for t in trg_string_list
      ]

In [186]:
trg_file_path = '../data/ptb/test.conllu'

with open(trg_file_path, 'r') as trg_file:
    trg_string = trg_file.read().strip()
    trg_string_list = trg_string.split('\n\n')
    try:
      trg_list = [
          DependencyGraph(t, top_relation_label='root') for t in trg_string_list
      ]
    except ValueError:

      def extract_10_cells(cells, index):
        line_index, word, lemma, tag, _, head, rel, _, _, _ = cells
        try:
          index = int(line_index)
        except ValueError:
          # index can't be parsed as an integer, use default
          pass
        return index, word, lemma, tag, tag, '', head, rel

      trg_list = [
          DependencyGraph(
              t, top_relation_label='root', cell_extractor=extract_10_cells)
          for t in trg_string_list
      ]


all_indices = []
all_deps = []
c = 0
for j in range(len(trg_list)):
    included_indices = []
    deps = []
    nodes_addresses = []
    for i in range(len(trg_list[j].nodes)):
        if trg_list[j].nodes[i]['ctag'] in WORD_TAGS:
            node_address = trg_list[j].nodes[i]['address']
            node_word = trg_list[j].nodes[i]['word']
            nodes_addresses.append(node_address)
            included_indices.append(dict({node_address:node_word}))
    
    removed_nodes_parents = dict()
    for i in range(len(trg_list[j].nodes)):
        node_address = trg_list[j].nodes[i]['address']
        if node_address not in nodes_addresses and node_address != 0:
            head = trg_list[j].nodes[i]['head']
            if head in nodes_addresses:
                removed_nodes_parents[node_address] = head
            else:
                head_2 = trg_list[j].get_by_address(head)['head']
                if head_2 in nodes_addresses:
                    removed_nodes_parents[node_address] = head_2
                else:
                    c = c + 1
                    #print(f"MOSSEBA head1 {head} head2 {head_2} node_address {node_address} in {j}")
                    #print(trg_list[j].to_conll(10))
                    #raise AssertionError
    
    # for i in range(len(trg_list[j].nodes)):
    #     node_address = trg_list[j].nodes[i]['address']
    #     if node_address in nodes_addresses:
    #         head = trg_list[j].nodes[i]['head']
    #         if head in nodes_addresses:
    #             deps.append([node_address, head])
    #         elif head == 0:
    #             deps.append([node_address, head])
    #         else: # head is not in nodes_addresses, parent of node got filtered out
    #             deps.append([node_address, removed_nodes_parents[head]])
    #             #raise AssertionError(f"in {j}, head {head} is not in nodes_addresses {nodes_addresses} in {trg_list[j].nodes}")
            
    
    
    # set_all_indices = set()
    # for item in included_indices:
    #     set_all_indices.add(list(item.keys())[0])
    # set_all_indices.add(0)
    # set_all_deps = set()
    # for item in deps:
    #     set_all_deps.add(item[0])
    #     set_all_deps.add(item[1])
    
    # if set_all_indices != set_all_deps:
    #     print(included_indices)
    #     print(deps)
    #     print(set_all_indices)
    #     print(set_all_deps)
    #     print()
    #     raise AssertionError(f"at 1 seneence {j}")
    
    # all_indices.append(included_indices)
    # all_deps.append(deps)

# k = 0
# for indices, deps in zip(all_indices, all_deps):
#     assert len(all_indices[k]) == len(all_deps[k]), f"at 1: length of indices and deps are not equal for sentence {k}"
#     l1 = len(all_indices[k])
#     m1 = copy.deepcopy(all_indices[k])
#     m2 = copy.deepcopy(all_deps[k])
#     removed_indices = []
#     removed_deps = []
#     flag = False
#     for i, ind in enumerate(indices):
#         token = list(ind.values())[0] # get the token
#         if token.lower() in NEG_POSS:
#             index = list(ind.keys())[0]
#             # merge the token with the previous token
#             all_indices[k][i-1][list(indices[i-1].keys())[0]] += token
#             # remove the token from the indices
#             removed_indices.append(ind)
            
#             # get the head of token
#             head = None
#             for j, dep in enumerate(deps):
#                 if dep[0] == index:
#                     head = dep[1]
#                     break
            
#             # update the dependencies
#             for j, dep in enumerate(deps):
#                 if dep[0] == index:
#                     removed_deps.append(dep)
#                 if dep[1] == index:
#                     flag = True
#                     if all_deps[k][j][0] != list(all_indices[k][i-1].keys())[0]:
#                         all_deps[k][j][1] = list(all_indices[k][i-1].keys())[0]
#                     else: # if the head of the token is the token itself
#                         all_deps[k][j][1] = head
            
#     for item in removed_indices:
#         all_indices[k].remove(item)
#     for item in removed_deps:
#         all_deps[k].remove(item)            
#     try:
#         assert len(all_indices[k]) == len(all_deps[k]), f"at 2: length of indices and deps are not equal for sentence {k}"
#     except AssertionError as e:
#         print(k)
#         print(all_indices[k])
#         print(all_deps[k])
#         print(len(all_indices[k]), len(all_deps[k]))
#         print()
#         raise AssertionError(f"{e} at sentence {k}")
#     l2 = len(all_indices[k])
    
#     set_all_indices = set()
#     for item in all_indices[k]:
#         set_all_indices.add(list(item.keys())[0])
#     set_all_indices.add(0)
#     set_all_deps = set()
#     for item in all_deps[k]:
#         set_all_deps.add(item[0])
#         set_all_deps.add(item[1])
    
#     if set_all_indices != set_all_deps:
#         print(k)
#         print(m1)
#         print(all_indices[k])
#         print(m2)
#         print(all_deps[k])
#         print(set_all_indices)
#         print(set_all_deps)
#         print()
#         raise AssertionError(f"at sentence {k}")
    
#     if False:
#         print(k)
#         print(m1)
#         print(all_indices[k])
#         print(m2)
#         print(all_deps[k])
#         print(l1, l2)
#         print("=====================================")
#     k += 1

In [187]:
c

57

In [182]:
print(trg_list[194].to_conll(10))

1	A	a	DT	DT	_	2	det	_	_
2	spokesman	spokesman	NN	NN	_	3	nsubj	_	_
3	added	add	VBD	VBD	_	0	root	_	_
4	that	that	IN	IN	_	12	mark	_	_
5	sales	sale	NNS	NNS	_	12	nsubj	_	_
6	in	in	IN	IN	_	9	case	_	_
7	the	the	DT	DT	_	9	det	_	_
8	current	current	JJ	JJ	_	9	amod	_	_
9	quarter	quarter	NN	NN	_	5	nmod	_	_
10	will	will	NN	NN	_	12	dep	_	_
11	about	about	RB	RB	_	12	advmod	_	_
12	equal	equal	VB	VB	_	3	ccomp	_	_
13	the	the	DT	DT	_	15	det	_	_
14	yearearlier	yearearlier	JJ	JJ	_	15	amod	_	_
15	quarter	quarter	NN	NN	_	17	nmod:poss	_	_
16	's	's	POS	POS	_	15	case	_	_
17	figure	figure	NN	NN	_	12	dobj	_	_
18	,	,	,	,	_	17	punct	_	_
19	when	when	WRB	WRB	_	21	advmod	_	_
20	Newport	Newport	NNP	NNP	_	21	nsubj	_	_
21	reported	report	VBD	VBD	_	17	acl:relcl	_	_
22	net	net	JJ	JJ	_	23	amod	_	_
23	income	income	NN	NN	_	21	dobj	_	_
24	of	of	IN	IN	_	25	case	_	_
25	$	$	$	$	_	23	nmod	_	_
26	1.7	1.7	CD	CD	_	27	compound	_	_
27	million	million	CD	CD	_	25	nummod	_	_
28	,	,	,	,	_	25	punct	_	_
29	or	or	CC	CC	_	25	cc	_	_
30	21	21	

In [143]:
trg_list[21].nodes[1]

{'address': 1,
 'word': 'Shares',
 'lemma': 'Shares',
 'ctag': 'NNP',
 'tag': 'NNP',
 'feats': '_',
 'head': 13,
 'deps': defaultdict(list, {'nmod': [3]}),
 'rel': 'nsubj'}

In [142]:
print(trg_list[21].to_conll(10))

1	Shares	Shares	NNP	NNP	_	13	nsubj	_	_
2	of	of	IN	IN	_	3	case	_	_
3	UAL	UAL	NNP	NNP	_	1	nmod	_	_
4	,	,	,	,	_	3	punct	_	_
5	the	the	DT	DT	_	6	det	_	_
6	parent	parent	NN	NN	_	3	appos	_	_
7	of	of	IN	IN	_	9	case	_	_
8	United	United	NNP	NNP	_	9	compound	_	_
9	Airlines	Airlines	NNP	NNP	_	6	nmod	_	_
10	,	,	,	,	_	3	punct	_	_
11	were	be	VBD	VBD	_	13	cop	_	_
12	extremely	extremely	RB	RB	_	13	advmod	_	_
13	active	active	JJ	JJ	_	0	root	_	_
14	all	all	DT	DT	_	15	det	_	_
15	day	day	NN	NN	_	13	nmod:tmod	_	_
16	Friday	Friday	NNP	NNP	_	13	nmod:tmod	_	_
17	,	,	,	,	_	13	punct	_	_
18	reacting	react	VBG	VBG	_	13	advcl	_	_
19	to	to	TO	TO	_	20	case	_	_
20	news	news	NN	NN	_	18	nmod	_	_
21	and	and	CC	CC	_	20	cc	_	_
22	rumors	rumor	NNS	NNS	_	20	conj	_	_
23	about	about	IN	IN	_	29	case	_	_
24	the	the	DT	DT	_	29	det	_	_
25	proposed	propose	VBN	VBN	_	29	amod	_	_
26	$	$	$	$	_	29	amod	_	_
27	6.79	6.79	CD	CD	_	28	compound	_	_
28	billion	billion	CD	CD	_	26	nummod	_	_
29	buy-out	buy-out	NN	NN	_	20	nmod	_	_
30	of	of	IN	I

In [133]:
seta = set()
setb = set()

seta.add(3)
seta.add(1)
seta.add(2)
seta.add(3)

setb.add(1)
setb.add(2)
setb.add(3)


In [134]:
seta, setb

({1, 2, 3}, {1, 2, 3})

In [135]:
seta == setb

True

In [102]:
all_indices[0].remove({7: 'Monday'})

In [103]:
all_indices[0]

[{1: 'No'}, {3: 'it'}, {4: "wasn't"}, {6: 'Black'}]

In [52]:
a.remove([1,2])

ValueError: list.remove(x): x not in list

In [ ]:
original_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
indices_to_remove = [2, 5, 8]

new_list = [item for i, item in enumerate(original_list) if i not in indices_to_remove]

print(new_list)


In [5]:
loaded_model =  torch.load('../trained_models/1111_subword.pt')[0]

In [ ]:
my_list.remove(value)


In [19]:
def recall_subword_constituents(model_out, std_out, sen):
  true_subword_constituents = []
  for const in std_out:
      c = sen[const[0]:const[1]]
      try:
        if not c[0].startswith('Ġ'):
            flag=True
            for token in c:
                if token.startswith('Ġ'):
                    flag=False
                    break
            if flag:
                print(const)
                print(c)
                true_subword_constituents.append(const)
        else:
            flag=True
            for token in c[1:]:
                if token.startswith('Ġ'):
                    flag=False
                    break
            if flag:
                print(const)
                print(c)
                true_subword_constituents.append(const)
      except Exception as e:
        print("problem sent: ", sen)
        raise Exception("Error: ", e)
  
  total_subword_constituents = len(true_subword_constituents)
  correct_subword_constituents = 0
  
  print("true_subword_constituents: ", true_subword_constituents)
  print("model_out: ", model_out)
  
  for cons in true_subword_constituents:
      if cons in model_out:
        correct_subword_constituents += 1
      else:
        print("uncaught_constituents: ", sen[cons[0]:cons[1]])
        
  if total_subword_constituents == 0:
    return 1.0
  else:
    return correct_subword_constituents/total_subword_constituents

In [21]:
for i in range(10, 20):
    sen = ptb_corpus.tokenizer.tokenize(ptb_corpus.test_sens[i])
    x = ptb_corpus.test[i]

    data = torch.LongTensor([x]).to(device)
    pos = torch.LongTensor([list(range(len(x)))]).to(device)

    _, p_dict = loaded_model(data, pos)
    block = p_dict['block']
    cibling = p_dict['cibling']
    head = p_dict['head']
    distance = p_dict['distance']
    height = p_dict['height']

    distance = distance.detach().clone().squeeze(0).cpu().numpy().tolist()
    height = height.detach().clone().squeeze(0).cpu().numpy().tolist()
    head = head.detach().clone().squeeze(0).cpu().numpy()
    max_height = np.max(height)

    parse_tree = tree_utils.build_tree(distance, sen)

    model_out, _ = tree_utils.get_brackets(parse_tree)
    std_out, _ = tree_utils.get_brackets(ptb_corpus.test_trees[i])
    overlap = model_out.intersection(std_out)

    prec = float(len(overlap)) / (len(model_out) + 1e-8)
    reca = float(len(overlap)) / (len(std_out) + 1e-8)
    if not std_out:
        reca = 1.
        if not model_out:
            prec = 1.
    f1 = 2 * prec * reca / (prec + reca + 1e-8)
    subword_reca = recall_subword_constituents(model_out, std_out, sen)
    print(f"f1: {f1:.4f}, prec: {prec:.4f}, reca: {reca:.4f}, subword_reca: {subword_reca:.4f}")
    print("=========================================")

(22, 24)
['Ġisn', "'t"]
(13, 15)
['Ġinc', '.']
(3, 6)
['Ġmag', 'u', 'ire']
(0, 2)
['count', 'ered']
(9, 12)
['Ġhen', 'd', 'erson']
true_subword_constituents:  [(22, 24), (13, 15), (3, 6), (0, 2), (9, 12)]
model_out:  {(20, 27), (15, 27), (2, 6), (17, 19), (20, 22), (22, 24), (13, 15), (24, 26), (16, 19), (3, 5), (2, 7), (9, 13), (7, 15), (9, 15), (22, 27), (24, 27), (1, 27), (19, 27), (16, 27), (9, 11), (9, 12), (3, 6), (2, 15), (2, 27), (8, 15)}
uncaught_constituents:  ['count', 'ered']
f1: 0.6818, prec: 0.6000, reca: 0.7895, subword_reca: 0.8000
(6, 9)
['Ġfree', '-', 'fall']
(12, 14)
['Ġcan', "'t"]
true_subword_constituents:  [(6, 9), (12, 14)]
model_out:  {(4, 10), (5, 9), (1, 3), (6, 9), (3, 12), (4, 9), (6, 8), (12, 15), (1, 16), (3, 10), (13, 15), (10, 12), (12, 16), (3, 16)}
uncaught_constituents:  ['Ġcan', "'t"]
f1: 0.4000, prec: 0.3571, reca: 0.4545, subword_reca: 0.5000
(0, 3)
['spec', 'ul', 'ators']
true_subword_constituents:  [(0, 3)]
model_out:  {(4, 10), (5, 9), (1, 3), (

In [9]:
prec, reca, f1, subword_reca

(0.49999999875, 0.6666666644444444, 0.5714285648979592, 1.0)